<a href="https://colab.research.google.com/github/Roghiehghahremani/Deep-Learning-With-PyTorch---Full-Course_Patrick-Loeber/blob/main/14_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np


In [9]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/DLPyTorch_Patrick_Loeber

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/DLPyTorch_Patrick_Loeber


In [10]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [25]:
# Hyper=parameters
num_epochs = 20
batch_size = 4
learning_rate = 0.001


In [26]:
# dataset has PILImage images of range [0, 1]
# we transform them to Tensors of normalized range [-1,1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [27]:
train_dataset = torchvision.datasets.CIFAR10(root = './data',train=True,
                                             download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root = './data',train=False,
                                             download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,
                                           shuffle=False)
classes = ('plane',"car",'bird','cat',
           'deer','dog', 'frog','horse','ship','truck' )


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# implement conv net
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1,16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x



model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    # origin shape [4,3,32,32] 4, 3 , 1024
    # input_layers 3 input channels, 6 output channels , 5 kernal size
    images = images.to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs,labels)

    # backward pass and optimize
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    if (i+1) % 12500 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}, Step {i+1} / {n_total_steps}, Loss = {loss.item():.4f}]')
print('finished training')

Epoch [1/20, Step 12500 / 12500, Loss = 2.0763]
Epoch [2/20, Step 12500 / 12500, Loss = 1.3964]
Epoch [3/20, Step 12500 / 12500, Loss = 1.3038]
Epoch [4/20, Step 12500 / 12500, Loss = 2.0168]
Epoch [5/20, Step 12500 / 12500, Loss = 1.1949]
Epoch [6/20, Step 12500 / 12500, Loss = 0.7638]


In [24]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images,labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    # max returns value, index
    _,predicted = torch.max(outputs,1)
    n_samples += labels.shape[0]
    n_correct += (predicted == labels).sum().item()
    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1


  acc = 100.0 * n_correct/n_samples
  print(f'accuracy of the network= {acc}')
  for i in range(10):
    acc = 100.0 * n_class_correct[i]/n_class_samples[i]
    print(f'accuracy of {classes[i]}= {acc} %')

accuracy of the network= 59.01
accuracy of plane= 77.5 %
accuracy of car= 69.9 %
accuracy of bird= 27.6 %
accuracy of cat= 42.0 %
accuracy of deer= 55.6 %
accuracy of dog= 48.6 %
accuracy of frog= 81.2 %
accuracy of horse= 63.8 %
accuracy of ship= 61.4 %
accuracy of truck= 62.5 %
